# Analiza liczby pasażerów transportu kolejowego
#### Autorzy: Szymon Kowalski, Zuzanna Czyżowska - zespół 18
## 1. Wprowadzenie i cel analizy

W tym projekcie zajmiemy się analizą systemów kolejowych w państwach europejskich, w celu określenia czynników wpływających na roczną liczbę pasażerów transportu kolejowego w danym państwie. Korzystając z danych zebranych przez Eurostat w latach 2014-2023 dotyczących m.in. infrastruktury kolejowej, liczby pojazdów kolejowych, sytuacji ekonomicznej w kraju, wielkości inwestycji w transport kolejowy, spróbujemy zdefiniować najistotniejsze cechy, od których zależna jest liczba pasażerów kolei. Następnie na podstawie otrzymanych wyników stworzymy model regresji, mający na celu przewidywanie liczby użytkowników kolei w danym roku w danym kraju.

Źródło danych: https://ec.europa.eu/eurostat/databrowser/explore/all/transp?lang=en&subtheme=rail&display=list&sort=category

## 2. Hipoteza badawcza


## 3. Załadowanie danych

Import potrzebnych bibliotek:

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Wczytanie i przygotowanie danych:

In [40]:
tables = pd.read_excel('all_data.xlsx', sheet_name='Arkusz1', skiprows=1)
tables = tables.dropna(axis=0, how='all')
tables = tables.dropna(axis=1, how='all')

print("Dane:")
print(tables.head())
print("Rozmiar danych:")
print(tables.shape)

Dane:
   n  Year                                    Country pa_tr pa_tr_na pa_tr_in  \
0  1  2014  European Union - 27 countries (from 2020)     :        :        :   
1  2  2014  European Union - 28 countries (2013-2020)     :        :        :   
2  3  2014  European Union - 27 countries (2007-2013)     :        :        :   
3  4  2014                                    Belgium     :        :        :   
4  5  2014                                   Bulgaria  1698     1683       15   

     tr_tr le_ra_tr pa_ra_ve         po gdp_pc    ex_in in_in le_mo  co_pr_le  \
0      NaN        :        :  442266046  25660        :     :     :     100.0   
1  3309453        :        :  341548691      :        :     :     :     105.1   
2  3294324        :        :  337331059    NaN        :     :     :     105.4   
3        :     6250        :   11180840  33790        :     :  1763     111.2   
4    20453     6481        :    7117453   5590  387.616     :   610      49.4   

  in_ra ca_pa_ra_ve 

### Opis zbioru:
Każda obserwacja występuje dla danego europejskiego kraju lub wszystkich państw Unii Europejskiej w danym roku (od 2014 do 2023). W zbiorze widać, że jest 18 kolumn:
- n - numer obserwacji,
- Year - rok obserwacji (dane są zbierane na stan 1 stycznia danego roku)
- Country - kraj lub obserwacja ogólnie dla Unii Europejskiej
Nasza kolumna-klasa (jednostka):
- pa_tr - przewiezieni pasażerowie (mln pasażerokm - miara która określa ile pasażerów przebyło km)
Pozostałe kolumny to cechy, któryvh wpływ na nasza klasę będziemy sprawdzać:
- pa_tr_na - przewiezieni pasażerowie w transporcie krajowym (mln pasażerokm)
- pa_tr_in - przewiezieni pasażerowie w transporcie międzynarodowym (mln pasażerokm)
- tr_tr - praca przewozowa pociagów (tys. pociągokm - ile pociągów przebyło 1 km)
- le_ra_tr - długość torów kolejowych (km)
- pa_ra_ve - liczba pociągów pasażerskich (liczba)
- po - populacja kraju (liczba)
- gdp_pc - PKB per capita (euro)
- ex_in - wydatki na infrastrukturę (mln euro) - utrzymanie, konserwacja, naprawy itp.
- in_in - inwestycje w infrastrukturę (mln euro)
- le_mo - długość autostrad (km) - jako alternatywa dla kolei
- co_pr_le - porównawcze poziomy cen w danym kraju (wskaźnik poziomu cen, UE-27 = 100) - ta cecha ma na celu pokazanie, czy dany kraj jest droższy lub tańszy w porównaniu do średniej unijnej, poniżej 100 - jest tańszy, powyżej 100 - droższy
- in_ra - stopa inflacji (HICP - zharmonizowany wskaźnik cen konsumpcyjnych)
- ca_pa_ra_ve - pojemność pasażerskich pojazdów kolejowych (tysiące miejsc)
- le_hi_sp_ra - długość dedykowanych linii kolei dużych prędkości (kilometry)

## 4. Analiza jakości danych

Przekonwertowanie na odpowiednie typy danych.

In [41]:
tables[['n', 'Country']] = tables[['n', 'Country']].astype(str)
tables['Year'] = pd.to_datetime(tables['Year'], format='%Y')

columns_to_convert = tables.columns[3:]
for col in columns_to_convert:
    tables[col] = tables[col].astype(str).str.strip().str.replace(",", ".", regex=False)
    tables[col] = pd.to_numeric(tables[col], errors='coerce')

print(tables.dtypes)

n                      object
Year           datetime64[ns]
Country                object
pa_tr                 float64
pa_tr_na              float64
pa_tr_in              float64
tr_tr                 float64
le_ra_tr              float64
pa_ra_ve              float64
po                    float64
gdp_pc                float64
ex_in                 float64
in_in                 float64
le_mo                 float64
co_pr_le              float64
in_ra                 float64
ca_pa_ra_ve           float64
le_hi_sp_ra           float64
dtype: object


Sprawdzenie ile jest braków w danych

In [42]:
print("Braki danych: \n", tables.isnull().sum())

Braki danych: 
 n                0
Year             0
Country          0
pa_tr           52
pa_tr_na        66
pa_tr_in        69
tr_tr           57
le_ra_tr        69
pa_ra_ve       136
po               5
gdp_pc          25
ex_in          194
in_in          187
le_mo           52
co_pr_le         0
in_ra            6
ca_pa_ra_ve    162
le_hi_sp_ra     24
dtype: int64


Widzimy, że występuje wiele braków danych w tym zbiorze. Zanim, więc przejdziemy do dalszej analizy należy zdecydować co z nimi zrobić.

In [43]:
# Najpierw sprawdzimy, czy niektóre obserwacje nie mają za dużo braków, jeśli mają, wtedy usuniemy całe obserwacje
missing_in_row= tables.isnull().mean(axis=1)

# Filtr dla obserwacji z brakami powyżej określonego progu (np. 20%)
threshold = 0.3
rows_with_many_nans = tables[missing_in_row > threshold]

# Wyświetlenie wyników
print("Obserwacje z brakami danych powyżej 30%:")
for index, row in rows_with_many_nans.iterrows():
    print(f"Indeks: {index}, Rok: {row['Year']}, Kraj: {row['Country']}")
print(len(rows_with_many_nans))

Obserwacje z brakami danych powyżej 30%:
Indeks: 0, Rok: 2014-01-01 00:00:00, Kraj: European Union - 27 countries (from 2020)
Indeks: 1, Rok: 2014-01-01 00:00:00, Kraj: European Union - 28 countries (2013-2020)
Indeks: 2, Rok: 2014-01-01 00:00:00, Kraj: European Union - 27 countries (2007-2013)
Indeks: 3, Rok: 2014-01-01 00:00:00, Kraj: Belgium
Indeks: 19, Rok: 2014-01-01 00:00:00, Kraj: Netherlands
Indeks: 32, Rok: 2014-01-01 00:00:00, Kraj: Montenegro
Indeks: 35, Rok: 2015-01-01 00:00:00, Kraj: European Union - 27 countries (from 2020)
Indeks: 36, Rok: 2015-01-01 00:00:00, Kraj: European Union - 28 countries (2013-2020)
Indeks: 37, Rok: 2015-01-01 00:00:00, Kraj: European Union - 27 countries (2007-2013)
Indeks: 38, Rok: 2015-01-01 00:00:00, Kraj: Belgium
Indeks: 53, Rok: 2015-01-01 00:00:00, Kraj: Hungary
Indeks: 54, Rok: 2015-01-01 00:00:00, Kraj: Netherlands
Indeks: 67, Rok: 2015-01-01 00:00:00, Kraj: Montenegro
Indeks: 70, Rok: 2016-01-01 00:00:00, Kraj: European Union - 27 count

Ze względu na to, że niektóre państwa dla każdego roku mają duże braki danych, możemy z naszej analizy wyrzucić Belgię, Węgry, Holandię i Czarnogórę oraz wszystkie wiersze przedstawiające dane dla ogólnie Unii Europejskiej.

In [44]:
countries_to_remove = ['European Union - 27 countries (from 2020)', 'European Union - 28 countries (2013-2020)', 'European Union - 27 countries (2007-2013)', 'Belgium', 'Hungary', 'Netherlands', 'Montenegro']

tables_cleaned = tables[~tables['Country'].isin(countries_to_remove)]
print("Braki danych: \n", tables_cleaned.isnull().sum())

Braki danych: 
 n                0
Year             0
Country          0
pa_tr           10
pa_tr_na        14
pa_tr_in        17
tr_tr           16
le_ra_tr        39
pa_ra_ve        80
po               5
gdp_pc           5
ex_in          126
in_in          123
le_mo           14
co_pr_le         0
in_ra            4
ca_pa_ra_ve     96
le_hi_sp_ra      2
dtype: int64


Pomimo usunięcia wielu obserwacji nadal dla niektórych cech są duże braki danych.

In [45]:
missing_percentage = tables_cleaned.isnull().mean()
threshold = 0.3
columns_with_big_missing = missing_percentage[missing_percentage > threshold]
print(columns_with_big_missing)

ex_in          0.450000
in_in          0.439286
ca_pa_ra_ve    0.342857
dtype: float64


Niestety cechy, dla których wyszły największe braki mogą być bardzo istotne przy określeniu zależności liczby pasażerów, ponieważ są to wydatki na infrastrukturę, inwestycje oraz pojemność pociągów. W związku z tym nie będziemy usuwać tych cech, ale spróbujemy uzupełnić częściowo te braki.

In [38]:
# # Imputacja medianą
# print("Przed: \n", tables_cleaned[['ex_in', 'in_in']].describe())
# from sklearn.impute import SimpleImputer
#
# # Tworzymy imputera, który uzupełnia brakujące dane medianą
# imputer = SimpleImputer(strategy='median')
#
# # Imputacja danych
# tables_cleaned[['ex_in', 'in_in']] = imputer.fit_transform(tables_cleaned[['ex_in', 'in_in']])
# print("Po: \n", tables_cleaned[['ex_in', 'in_in']].describe())

# Inna opcja imputacja regresją
# from sklearn.linear_model import LinearRegression
#
# # Zbiór danych bez brakujących wartości
# train_data = tables_cleaned.dropna(subset=['ex_in'])  # Zbiór treningowy (bez braków w 'ex_in')
# X_train = train_data[['gdp_pc', 'po']]  # Cecha do treningu
# y_train = train_data['ex_in']  # Wartość do przewidywania
#
# # Tworzymy i trenujemy model regresji
# model = LinearRegression()
# model.fit(X_train, y_train)
#
# # Przewidywanie brakujących wartości
# missing_data = tables_cleaned[tables_cleaned['ex_in'].isnull()]
# X_missing = missing_data[['gdp_pc', 'po']]
# tables_cleaned.loc[tables_cleaned['ex_in'].isnull(), 'ex_in'] = model.predict(X_missing)


Przed: 
               ex_in          in_in
count    154.000000     157.000000
mean    1706.732974    4482.591369
std     2553.113609   37704.910662
min       14.316000       5.000000
25%      238.200000     180.000000
50%      553.963500     519.000000
75%     2232.461000    1797.176000
max    12969.801000  473252.000000
Po: 
               ex_in          in_in
count    280.000000     280.000000
mean    1187.986711    2741.442304
std     1976.025427   28262.878491
min       14.316000       5.000000
25%      485.405000     373.460000
50%      553.963500     519.000000
75%      746.391250     945.907500
max    12969.801000  473252.000000


/var/folders/vl/gs1w498x01bftxkwdxs2z5s80000gn/T/ipykernel_55162/3153850735.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tables_cleaned[['ex_in', 'in_in']] = imputer.fit_transform(tables_cleaned[['ex_in', 'in_in']])


Sprawdzenie czy są jakieś duplikaty

In [46]:
print("Duplikaty: \n", tables_cleaned.duplicated().sum())

Duplikaty: 
 0


## 5. Statystyki opisowe

Podstawowe statystyki dla cech liczbowych (średnia, mediana, odchylenie standardowe, min, max, kwartyle):

In [47]:
print(tables_cleaned.describe())

                      Year          pa_tr      pa_tr_na      pa_tr_in  \
count                  280     270.000000    266.000000    263.000000   
mean   2018-07-02 12:00:00   14241.988889  14104.259398    717.387833   
min    2014-01-01 00:00:00      14.000000     25.000000      0.000000   
25%    2016-01-01 00:00:00     631.000000    716.250000     21.000000   
50%    2018-07-02 12:00:00    3711.500000   3759.500000    100.000000   
75%    2021-01-01 00:00:00   12841.250000  12714.000000    490.500000   
max    2023-01-01 00:00:00  101408.000000  95453.000000  10810.000000   
std                    NaN   25219.483151  23770.814215   1721.010122   

               tr_tr      le_ra_tr      pa_ra_ve            po        gdp_pc  \
count     264.000000    241.000000    200.000000  2.750000e+02    275.000000   
mean   110089.753788  14299.256822   2870.305000  2.005383e+07  29178.654545   
min       597.000000      9.000000     65.000000  5.496800e+05   4030.000000   
25%      9543.000000  

Analiza cech kategorycznych (u nas tylko kraje):

In [48]:
print(tables_cleaned["Country"].value_counts())

Country
Bulgaria                  10
Czechia                   10
North Macedonia           10
Bosnia and Herzegovina    10
United Kingdom            10
Switzerland               10
Norway                    10
Sweden                    10
Finland                   10
Slovakia                  10
Slovenia                  10
Romania                   10
Portugal                  10
Poland                    10
Austria                   10
Luxembourg                10
Lithuania                 10
Latvia                    10
Italy                     10
Croatia                   10
France                    10
Spain                     10
Greece                    10
Ireland                   10
Estonia                   10
Germany                   10
Denmark                   10
Türkiye                   10
Name: count, dtype: int64


## 6. Wizualizacja danych

Rozkłady zmiennych, histogramy, wykresy rozrzutu, macierz korelacji, wykresy pudełkowe

## 7. Analiza zależności, korelacji i hipotez

Analiza korelacji, wartości odstające, hipotezy